# Creating the main_cell & JJ and JJ-Array

This notebook created by Alena Kazmina, Figen Yilmaz and Christian Kraglund Andersen

In [14]:
import gdspy
from importlib import reload
from itertools import product
reload(gdspy)
lib = gdspy.GdsLibrary()

#######################################################################################

#create base layer with main ground and markers
main_cell = lib.new_cell('main_cell')

unground = gdspy.Rectangle((-4500,-4500), (4500, 4500), layer=2)
main_cell.add(unground)

      
#logo and the device name
device_name = gdspy.Text("Your Device Name", 150, (800, -4350), layer=3)
logo = gdspy.Text("Andersen Lab/QuTech", 150, (250, -4500), layer=3)
main_cell.add(logo)
main_cell.add(device_name)

#######################################################################################

##### N SHOULD BE EVEN!!!!!!!!!!
# N is a number of JJ in the array
N = 100

if int(N / 2) % 2 != 0:
    raise ValueError('N / 2 should be even!!!')

delta_x = 0.5
# delta_x_base = 1.5
delta_x_base = 1.5
# nanowire_length is  the length between nanowire's pads
nanowire_length = 130 + 3
real_nanowire_length = nanowire_length - delta_x * 2 - delta_x_base * 2

# very important shift at x direction
SHIFT = - nanowire_length / 2
nanowire_width = 1.3

delta = 0.410  # initial width of pad

nu_cell = 0

start_x = 0
start_y = 0

stop_x = start_x + nanowire_length
stop_y = start_y

# for vertical and horizontallines
step = 1.2
delta_W = (real_nanowire_length - 2 * step) / (N / 2 - 1)
delta_H = 3
print(delta_W)

corr1 = 1.5
corr2 = 1

u = 1 #undercut length

if delta == 0.410:
    cell = lib.new_cell('jj_array' + str(nu_cell))
    
# Bandage for the left    
    ban1 = gdspy.Rectangle((start_x + delta_x_base - 2.5 + SHIFT - 0.2, start_y - 0.7 / 2 - 0.2-3),
                           (start_x + delta_x_base + delta_x + SHIFT + 0.2, start_y + 0.7 / 2 + 0.2-3),
                          layer=56)
    
    cell.add(ban1)
    
    # left line
    
    left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x+SHIFT-1.6-5.5, start_y - delta / 2-3),
                                (start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT-4.0, start_y + delta / 2-3), layer=54)
    cell.add(left_line)
    
    undercut_left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT-4.0, start_y - delta / 2-3),
                                         (start_x + delta_x_base + delta_x + step + delta+corr2+0.7+SHIFT-4.0, start_y + delta / 2-3),
                                         layer=55)
    cell.add(undercut_left_line)
    
# Bandage for the right    
    ban2 = gdspy.Rectangle((stop_x + SHIFT -1.8, stop_y - 3.7),
                           (stop_x + SHIFT -1.8+2.6, stop_y - 3.7 + 1.1), 
                          layer=56)
    cell.add(ban2)
    
    
########################### ADDING JJ ARRAY ########################################################### 
    array_JJ = None
    k = 0 #  center
    p = 0 #  down 
    for i in range(int(N / 2)):
        if i<1:
            start_y=-3
            delta_H = 3.8
            start_x = -7
            corr2 = 8
        elif i<2:
            start_y = -3
            delta_H = 3
            start_x = 0
            corr2 = 1
        else:
            start_y=0
            delta_H = 3
            start_x = 0
            corr2 = 1
        if i<N//2-1:
            vertical_pad = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y - 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6),
                layer=54)
            vertical_circle_1 = gdspy.Round((start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT +delta/2,
                                             start_y-5),
                                            radius=1.6*delta/2, layer=54, number_of_points=64)
            vertical_pad = gdspy.boolean(vertical_pad, vertical_circle_1, 'or', layer=54)
            vertical_circle_2 = gdspy.Round((start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT +delta/2,
                                             start_y+5),
                                            radius=1.6*delta/2, layer=54, number_of_points=64)
            vertical_pad = gdspy.boolean(vertical_pad, vertical_circle_2, 'or', layer=54)


            undercut_vertical = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
                 start_y + 6),
                (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
                 start_y + 6 + u*2),

                layer=55)

            array_JJ = gdspy.boolean(array_JJ, vertical_pad, 'or')

            cell.add(vertical_pad)
            cell.add(undercut_vertical)
        if i==N//2-2:
            corr2 = 2.5
        if i % 2 != 0 and i<N//2-1: #and i != int(N / 2) - 1:
            
            if k % 2 == 0:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+ 3 ),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+ 3),
                layer=55)
                
            else:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+1 ),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+1 ),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2 +1),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+1),
                layer=55)
                        
            k +=1
        
            array_JJ = gdspy.boolean(array_JJ, horizonatal_center, 'or')

            cell.add(horizonatal_center)

            cell.add(undercut_horizontal_center)

        if i % 2 == 0 and i<N//2-1:
            
            if p % 2 == 0:
                
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta),
                layer=55)
                
            else:
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta + 1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H + 1.5),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H+1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta+1.5),
                layer=55)
                
            p += 1

        
            array_JJ = gdspy.boolean(array_JJ, horizonatal_down, 'or')

            cell.add(horizonatal_down)

            cell.add(undercut_horizontal_center)
            
########################### ADDING JJ ###########################################################
            
deltaJJ = 0.120 # initial width of pad
pad_length_left = 2.4
pad_length_right = 1.7

nu_cellJJ = 0

if deltaJJ == 0.120:
            cell = lib.new_cell('junction_' + str(nu_cellJJ))
    
            left_big_points = [(-15., 0), (-10, 0), (-2., 1.5), (-2., 3), (-15, 3)]
            right_big_points = [(2., -1.5), (15, -1.5), (15., 1.5), (10, 1.5), (2., 0)]
    
            left_big = gdspy.Polygon(left_big_points, layer=53)
            right_big = gdspy.Polygon(right_big_points, layer=53)
    
            left_small = gdspy.Rectangle((-2., 1.5), (-0.2, 3), layer=53)
    
            right_small = gdspy.Rectangle((0.2, -1.5), (2., 0), layer=53)
    

        # Pads of junctions
            left_padJJ = gdspy.Rectangle((-1.2, -2.8), (-1.2 + deltaJJ, 2.3), layer=54)
    
            right_padJJ = gdspy.Rectangle((-3.0, -1.0), (1.2, -1.0 + deltaJJ), layer=54)
    
    
        # parts for undercuts
            undercut_left_pad3 = gdspy.Rectangle((-1.2, 2.3), (-1.2 + deltaJJ, + 3), layer=55)
    
            undercut_right_pad4 = gdspy.Rectangle((1.2, -1.0), (1.9, -1.0 + deltaJJ), layer=55)
    
        #bandages
            ban3 = gdspy.Rectangle((-1.8, 1.2), (-0.6, 3), layer=56)
            ban4 = gdspy.Rectangle((-0.25, -1.5), (2., -0.04), layer=56)
            

            cell.add(left_big)
            cell.add(right_big)

            cell.add(left_small)
            cell.add(right_small)

            cell.add(left_padJJ)
            cell.add(right_padJJ)
    
            cell.add(undercut_left_pad3)
            cell.add(undercut_right_pad4)
    
            cell.add(ban3)
            cell.add(ban4)
            
gdspy.LayoutViewer()

2.583673469387755


<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [2]:
## Your gds resource name
lib.write_gds('FX23_JJ120andArray410_NoMarkers_9by9mm.gds')

## You've completed your GDS cell resource file. Now, you need to use this GDS file to create the one containing all the JJs.